In [ ]:
from transformers import AutoTokenizer, BertModel
from torch.utils.data import TensorDataset, DataLoader
import sys
import torch
import numpy as np



if __name__ == "__main__":
    model = BertModel.from_pretrained("bert-base-uncased")
    sentence1, sentence2 = sys.argv[1:sys.argv.index(",")], sys.argv[sys.argv.index(",")+1:]

    sentence1, sentence2 = " ".join(sentence1), " ".join(sentence2)
    sentences = [sentence1, sentence2]

    # Tokenize each sentence
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
    sentences_tokenized = [tokenizer.tokenize(s) for s in sentences]
    MAX_LENGTH = max([len(sentences_tokenized[0]), len(sentences_tokenized[1])])

    # Generate IDs of each token and add padding to sentences smaller than given threshold
    ids = [tokenizer.convert_tokens_to_ids(t) for t in sentences_tokenized]
    # Pad the tokens
    ids = np.asarray([np.pad(i, (0, MAX_LENGTH-len(i)), mode='constant') for i in ids])


    # Generate the masks
    amasks = np.asarray([[float(i>0) for i in seq] for seq in ids])

    # Get the output from the model
    output = model.forward(torch.tensor(ids), torch.tensor(amasks))


    # Now create two vector representations for each sentence
    # pool_vectors (2x768) is the vector generated by perofrming max-pooling over the hidden states
    pool_vectors = torch.max(output[0],dim=1)[0]   # Fill up this '----------' section
    # cls_vectors (2x768) is the vector generated by taking the CLS token
    cls_vectors = output[0][:, 0, :]     # Fill up this '----------' section

    # Compute the cosine similarity
    cosine_pooling = torch.sum(pool_vectors[0] * pool_vectors[1]) / (torch.norm(pool_vectors[0]) * torch.norm(pool_vectors[1]))   # Fill up this '----------' section
    cosine_cls = torch.sum(cls_vectors[0] * cls_vectors[1]) / (torch.norm(cls_vectors[0]) * torch.norm(cls_vectors[1])) # Fill up this '----------' section

    # Finally print out the values
    print(np.round(cosine_pooling.item(), 2), np.round(cosine_cls.item(), 2))